# Flight Planning Domain 

This notebook aims to make a short and interactive example of the Flight Planning Domain. See the [online documentation](https://airbus.github.io/scikit-decide/reference/_skdecide.hub.domain.flight_planning.domain.html#flightplanningdomain) for more information.

In [ ]:
import datetime

from skdecide.hub.domain.flight_planning.domain import FlightPlanningDomain, WeatherDate
from skdecide.hub.solver.astar import Astar

## Definition of the problem

Here we will make a short haul flight from Paris Charles de Gaulle airport (ICAO : LFPG) to Toulouse-Blagnac airport (ICAO: LFBO), using an airbus A320 aircraft.

In [ ]:
origin = "LFPG"
destination = "LFBO"
aircraft = "A320"

Now, we are going to define a date that will be used for weather interpolation. If the data has not already be downloaded on your computer, be careful to choose a date within the past 6 months, so that the data is available on https://www.ncei.noaa.gov.

In [ ]:
# we set a date valid for 4 months to avoid downloading weather data at each daily run.
today = datetime.date.today()
month = ((today.month) - 1) // 4 * 4 + 1  # will result in january, may, or september
year = today.year
day = 1

weather_date = WeatherDate(day=day, month=month, year=year)
print(weather_date)

We can now define heuristic and cost function, to feed the A* solver. This aims to guide him along the airways graph to solve the problem, and get as close as possible to the optimal solution. 

The heuristic function can be either `"fuel"`, `"time"`, `"distance"`, `"lazy_fuel"`, `"lazy_time"`, or `None`. If `None`, the A* will behave as a Dijkstra-like search algorithm, as we give a 0 value to the A* algorithm. 

The cost function can be either `"fuel"`, `"time"`, or `"distance"`. 
It will define the cost of the flight plan, computed during the state-to-state flight simulation. 

In [ ]:
heuristic = "lazy_fuel"
cost_function = "fuel"

## Definition of the corresponding domain

We are now going to define the domain. It can take some time, especially if it needs to download some weather files, or if you ask for a fuel loop. 


In [ ]:
domain_factory = lambda: FlightPlanningDomain(
    origin,
    destination,
    aircraft,
    weather_date=weather_date,
    heuristic_name=heuristic,
    objective=cost_function,
    fuel_loop=False,
    graph_width="normal",
)
domain = domain_factory()

## Solving and rendering out the flight planning

We use her an A* solver as mentionned before. 
We also use the custom rollout proposed to have some visualization during the flight planning generation.

In [ ]:
with Astar(
    heuristic=lambda d, s: d.heuristic(s), domain_factory=domain_factory
) as solver:
    domain.solve_with(solver=solver, domain_factory=domain_factory)
    domain.custom_rollout(solver=solver)